In [1]:
# import packages
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ionq import IonQProvider

provider = IonQProvider(token='vm05jEWtO7zVGE6laZPtx8jbLM66TaS9')
backend = provider.get_backend("ionq_simulator")

In [2]:
def QuantumCircuitGen(state=None):
    three_d_markov = QuantumCircuit(5, 3)
    if state is not None:
        if state[0] == 1:
            three_d_markov.x(0)
        if state[1] == 1: 
            three_d_markov.x(1)
        if state[2] == 1:
            three_d_markov.x(2)
        
    three_d_markov.h(3)
    three_d_markov.h(4)
    three_d_markov.cnot(4, 0)
    three_d_markov.x(4)
    three_d_markov.cnot(4, 1)
    three_d_markov.cnot(3, 2)
    three_d_markov.toffoli(4, 3, 1)
    three_d_markov.x(4)
    three_d_markov.toffoli(4, 3, 0)
    three_d_markov.x(4)
    three_d_markov.toffoli(4, 3, 2)
    three_d_markov.measure([0, 1, 2], [0, 1, 2])
    
    return three_d_markov


In [25]:
three_d_markov.draw()


┌───┐                       ┌───┐     ┌─┐        
q_0: ─────┤ X ├───────────────────────┤ X ├─────┤M├────────
          └─┬─┘     ┌───┐┌───┐     ┌─┐└─┬─┘     └╥┘        
q_1: ───────┼───────┤ X ├┤ X ├─────┤M├──┼────────╫─────────
            │  ┌───┐└─┬─┘└─┬─┘     └╥┘  │        ║ ┌───┐┌─┐
q_2: ───────┼──┤ X ├──┼────┼────────╫───┼────────╫─┤ X ├┤M├
     ┌───┐  │  └─┬─┘  │    │        ║   │        ║ └─┬─┘└╥┘
q_3: ┤ H ├──┼────■────┼────■────────╫───■────────╫───■───╫─
     ├───┤  │  ┌───┐  │    │  ┌───┐ ║   │  ┌───┐ ║   │   ║ 
q_4: ┤ H ├──■──┤ X ├──■────■──┤ X ├─╫───■──┤ X ├─╫───■───╫─
     └───┘     └───┘          └───┘ ║      └───┘ ║       ║ 
c: 3/═══════════════════════════════╩════════════╩═══════╩═
                                    1            0       2

In [12]:
results_dic = {'000': 1, '100': 0, '010': 0, '001': 0, '110': 0, 
           '101': 0, '011': 0, '111': 0}

last_state = None
counter = 8
for step in range(0, counter):
    print('this is the input state: {}'.format(last_state))
    print(f'Steps left: {counter}')
    if step == 0:
        circuit = QuantumCircuitGen()
    else:
        circuit = QuantumCircuitGen(last_state)
        # print(circuit)
    job = backend.run(circuit, shots=1)
    result = job.result()
    counts = result.get_counts()
    print(counts)
    for key in counts:
        if counts[key] == 1:
            last_state = [int(s) for s in reversed(key)]
            break
    results_dic[key] += 1
    counter -= 1

print(results_dic)

        
    


this is the input state: None
Steps left: 8
{'000': 0, '001': 0, '010': 0, '100': 1}
this is the input state: [0, 0, 1]
Steps left: 7
{'000': 0, '100': 1, '101': 0, '110': 0}
this is the input state: [0, 0, 1]
Steps left: 6
{'000': 1, '100': 0, '101': 0, '110': 0}
this is the input state: [0, 0, 0]
Steps left: 5
{'000': 1, '001': 0, '010': 0, '100': 0}
this is the input state: [0, 0, 0]
Steps left: 4
{'000': 0, '001': 1, '010': 0, '100': 0}
this is the input state: [1, 0, 0]
Steps left: 3
{'000': 0, '001': 1, '011': 0, '101': 0}
this is the input state: [1, 0, 0]
Steps left: 2
{'000': 0, '001': 0, '011': 1, '101': 0}
this is the input state: [1, 1, 0]
Steps left: 1
{'001': 0, '010': 0, '011': 1, '111': 0}
{'000': 3, '100': 2, '010': 0, '001': 2, '110': 0, '101': 0, '011': 2, '111': 0}


In [45]:
good_circuit = QuantumCircuitGen([0,1,1])
good_circuit.draw()
# job = backend.run(good_circuit, shots=2)

┌───┐                       ┌───┐     ┌─┐        
q_0: ─────┤ X ├───────────────────────┤ X ├─────┤M├────────
     ┌───┐└─┬─┘     ┌───┐┌───┐     ┌─┐└─┬─┘     └╥┘        
q_1: ┤ X ├──┼───────┤ X ├┤ X ├─────┤M├──┼────────╫─────────
     ├───┤  │  ┌───┐└─┬─┘└─┬─┘     └╥┘  │        ║ ┌───┐┌─┐
q_2: ┤ X ├──┼──┤ X ├──┼────┼────────╫───┼────────╫─┤ X ├┤M├
     ├───┤  │  └─┬─┘  │    │        ║   │        ║ └─┬─┘└╥┘
q_3: ┤ H ├──┼────■────┼────■────────╫───■────────╫───■───╫─
     ├───┤  │  ┌───┐  │    │  ┌───┐ ║   │  ┌───┐ ║   │   ║ 
q_4: ┤ H ├──■──┤ X ├──■────■──┤ X ├─╫───■──┤ X ├─╫───■───╫─
     └───┘     └───┘          └───┘ ║      └───┘ ║       ║ 
c: 3/═══════════════════════════════╩════════════╩═══════╩═
                                    1            0       2

In [42]:
result = job.result()
counts = result.get_counts()
print(counts)

probabilities = result.get_probabilities()
print(probabilities)

{'010': 0, '100': 0, '110': 0, '111': 2}
{'010': 0.25, '100': 0.25, '110': 0.25, '111': 0.25}


In [23]:
a = {'b': 3}
print(list(a.keys()))

['b']
